# Set up the environment

In [14]:
# Check if GPU is available
!nvidia-smi

Mon Dec  9 02:25:46 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [15]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Clone YOLOv5 repository and install dependencies

In [16]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!pip install -r requirements.txt

Cloning into 'yolov5'...
remote: Enumerating objects: 17075, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (41/41), done.
Receiving objects: 100% (17075/17075), 15.69 MiB | 1.01 MiB/s, done.
remote: Total 17075 (delta 27), reused 26 (delta 12), pack-reused 17022 (from 1)
Resolving deltas: 100% (11721/11721), done.
/content/yolov5/yolov5


# Download and prepare the GTSRB dataset

In [17]:
!pwd  # Print current working directory

/content/yolov5/yolov5


In [18]:
# Download the dataset
!wget https://sid.erda.dk/public/archives/daaeac0d7ce1152aea9b61d9f1e19370/GTSRB_Final_Training_Images.zip
!wget https://sid.erda.dk/public/archives/daaeac0d7ce1152aea9b61d9f1e19370/GTSRB_Final_Test_Images.zip
!wget https://sid.erda.dk/public/archives/daaeac0d7ce1152aea9b61d9f1e19370/GTSRB_Final_Test_GT.zip

# Unzip the files
!unzip -q GTSRB_Final_Training_Images.zip -d GTSRB
!unzip -q GTSRB_Final_Test_Images.zip -d GTSRB
!unzip -q GTSRB_Final_Test_GT.zip -d GTSRB

--2024-12-09 02:26:07--  https://sid.erda.dk/public/archives/daaeac0d7ce1152aea9b61d9f1e19370/GTSRB_Final_Training_Images.zip
Resolving sid.erda.dk (sid.erda.dk)... 130.225.104.13
Connecting to sid.erda.dk (sid.erda.dk)|130.225.104.13|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 276294756 (263M) [application/zip]
Saving to: ‘GTSRB_Final_Training_Images.zip’

GTSRB_Final_Trainin 100%[===================>] 263.50M   118MB/s    in 2.2s    

2024-12-09 02:26:09 (118 MB/s) - ‘GTSRB_Final_Training_Images.zip’ saved [276294756/276294756]

--2024-12-09 02:26:09--  https://sid.erda.dk/public/archives/daaeac0d7ce1152aea9b61d9f1e19370/GTSRB_Final_Test_Images.zip
Resolving sid.erda.dk (sid.erda.dk)... 130.225.104.13
Connecting to sid.erda.dk (sid.erda.dk)|130.225.104.13|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 88978620 (85M) [application/zip]
Saving to: ‘GTSRB_Final_Test_Images.zip’

GTSRB_Final_Test_Im 100%[===================>]  84.86

# Prepare the dataset in YOLOv5 format

In [19]:
import os
import csv
from PIL import Image
import glob
import shutil

def convert_to_yolo_format(input_folder, output_folder, split_ratio=0.8):
    # Create directory structure
    os.makedirs(f"{output_folder}/images/train", exist_ok=True)
    os.makedirs(f"{output_folder}/images/val", exist_ok=True)
    os.makedirs(f"{output_folder}/labels/train", exist_ok=True)
    os.makedirs(f"{output_folder}/labels/val", exist_ok=True)

    # Get all class folders
    class_folders = glob.glob(os.path.join(input_folder, '*'))

    for class_id, class_folder in enumerate(sorted(class_folders)):
        if not os.path.isdir(class_folder):
            continue

        # Get all images in the class folder
        images = [f for f in os.listdir(class_folder) if f.endswith('.ppm')]
        split_idx = int(len(images) * split_ratio)

        # Process each image
        for idx, img_name in enumerate(images):
            try:
                # Read image and get dimensions
                img_path = os.path.join(class_folder, img_name)
                img = Image.open(img_path)
                w, h = img.size

                # Read annotation from CSV
                csv_path = os.path.join(class_folder, f'GT-{os.path.basename(class_folder)}.csv')
                with open(csv_path, 'r') as csvfile:
                    reader = csv.reader(csvfile, delimiter=';')
                    next(reader)  # Skip header
                    for row in reader:
                        if row[0] == img_name:
                            x1, y1, x2, y2 = map(int, row[3:7])
                            break

                # Convert to YOLO format
                x_center = (x1 + x2) / (2.0 * w)
                y_center = (y1 + y2) / (2.0 * h)
                width = (x2 - x1) / w
                height = (y2 - y1) / h

                # Determine train or val split
                subset = "train" if idx < split_idx else "val"

                # Save converted image
                new_img_path = os.path.join(output_folder, 'images', subset, f'{class_id}_{img_name[:-4]}.jpg')
                img = img.convert('RGB')
                img.save(new_img_path)

                # Save label
                label_path = os.path.join(output_folder, 'labels', subset, f'{class_id}_{img_name[:-4]}.txt')
                with open(label_path, 'w') as f:
                    f.write(f"{class_id} {x_center} {y_center} {width} {height}\n")

            except Exception as e:
                print(f"Error processing {img_name}: {str(e)}")

# Run the conversion
input_path = '/content/yolov5/GTSRB/GTSRB/Final_Training/Images'
output_path = '/content/yolov5/GTSRB_YOLO'
convert_to_yolo_format(input_path, output_path)

# Verify the conversion
print("\nChecking directory structure:")
!ls -R /content/yolov5/GTSRB_YOLO

Streaming output truncated to the last 5000 lines.
1_00044_00007.txt   15_00019_00029.txt	25_00024_00025.txt  36_00008_00029.txt	7_00001_00007.txt
1_00044_00008.txt   15_00020_00000.txt	25_00024_00026.txt  36_00009_00000.txt	7_00001_00008.txt
1_00044_00012.txt   15_00020_00001.txt	25_00024_00027.txt  36_00009_00001.txt	7_00001_00011.txt
1_00044_00013.txt   15_00020_00002.txt	25_00024_00028.txt  36_00009_00002.txt	7_00001_00012.txt
1_00044_00015.txt   15_00020_00003.txt	25_00025_00000.txt  36_00009_00003.txt	7_00001_00014.txt
1_00044_00016.txt   15_00020_00004.txt	25_00025_00001.txt  36_00009_00004.txt	7_00001_00016.txt
1_00044_00018.txt   15_00020_00005.txt	25_00025_00002.txt  36_00009_00006.txt	7_00001_00017.txt
1_00044_00019.txt   15_00020_00006.txt	25_00025_00004.txt  36_00009_00007.txt	7_00001_00018.txt
1_00044_00021.txt   15_00020_00008.txt	25_00025_00005.txt  36_00009_00008.txt	7_00001_00020.txt
1_00044_00023.txt   15_00020_00009.txt	25_00025_00006.txt  36_00009_00010.txt	7_00001

# Create a script to reduce the dataset size:

In [20]:
# import os
# import random
# import shutil

# # Define paths
# original_train_dir = '/content/yolov5/GTSRB_YOLO/images/train'
# original_val_dir = '/content/yolov5/GTSRB_YOLO/images/val'
# original_train_labels_dir = '/content/yolov5/GTSRB_YOLO/labels/train'
# original_val_labels_dir = '/content/yolov5/GTSRB_YOLO/labels/val'

# # Create new directories for reduced dataset
# reduced_train_dir = '/content/yolov5/GTSRB_YOLO_reduced/images/train'
# reduced_val_dir = '/content/yolov5/GTSRB_YOLO_reduced/images/val'
# reduced_train_labels_dir = '/content/yolov5/GTSRB_YOLO_reduced/labels/train'
# reduced_val_labels_dir = '/content/yolov5/GTSRB_YOLO_reduced/labels/val'

# # Create directories
# os.makedirs(reduced_train_dir, exist_ok=True)
# os.makedirs(reduced_val_dir, exist_ok=True)
# os.makedirs(reduced_train_labels_dir, exist_ok=True)
# os.makedirs(reduced_val_labels_dir, exist_ok=True)

# def reduce_dataset(original_dir, original_labels_dir, reduced_dir, reduced_labels_dir):
#     images = os.listdir(original_dir)
#     reduced_count = int(len(images) * 0.1)  # Take 10%
#     selected_images = random.sample(images, reduced_count)

#     for image in selected_images:
#         # Copy image
#         shutil.copy(os.path.join(original_dir, image), os.path.join(reduced_dir, image))
#         # Copy corresponding label
#         label_file = image.replace('.jpg', '.txt')
#         if os.path.exists(os.path.join(original_labels_dir, label_file)):
#             shutil.copy(os.path.join(original_labels_dir, label_file),
#                        os.path.join(reduced_labels_dir, label_file))

# # Reduce both train and validation sets
# reduce_dataset(original_train_dir, original_train_labels_dir,
#               reduced_train_dir, reduced_train_labels_dir)
# reduce_dataset(original_val_dir, original_val_labels_dir,
#               reduced_val_dir, reduced_val_labels_dir)

# Create dataset configuration file

In [21]:
%%writefile /content/yolov5/data/gtsrb.yaml
train: /content/yolov5/GTSRB_YOLO/images/train
val: /content/yolov5/GTSRB_YOLO/images/val

nc: 43
names: ['20_speed', '30_speed', '50_speed', '60_speed', '70_speed', '80_speed', '80_lifted', '100_speed', '120_speed', 'no_overtaking_general', 'no_overtaking_trucks', 'right_of_way_crossing', 'right_of_way_general', 'give_way', 'stop', 'no_traffic_both', 'no_trucks', 'no_entry', 'danger', 'bend_left', 'bend_right', 'bend', 'uneven_road', 'slippery_road', 'road_narrows', 'construction', 'traffic_signal', 'pedestrian_crossing', 'school_crossing', 'cycles_crossing', 'snow', 'animals', 'restriction_ends', 'go_right', 'go_left', 'go_straight', 'go_right_or_straight', 'go_left_or_straight', 'keep_right', 'keep_left', 'roundabout', 'restriction_ends_overtaking', 'restriction_ends_overtaking_trucks']

Overwriting /content/yolov5/data/gtsrb.yaml


In [22]:
# %%writefile /content/yolov5/data/gtsrb.yaml
# train: /content/yolov5/GTSRB_YOLO_reduced/images/train
# val: /content/yolov5/GTSRB_YOLO_reduced/images/val

# nc: 43
# names: ['20_speed', '30_speed', '50_speed', '60_speed', '70_speed', '80_speed', '80_lifted', '100_speed', '120_speed', 'no_overtaking_general', 'no_overtaking_trucks', 'right_of_way_crossing', 'right_of_way_general', 'give_way', 'stop', 'no_traffic_both', 'no_trucks', 'no_entry', 'danger', 'bend_left', 'bend_right', 'bend', 'uneven_road', 'slippery_road', 'road_narrows', 'construction', 'traffic_signal', 'pedestrian_crossing', 'school_crossing', 'cycles_crossing', 'snow', 'animals', 'restriction_ends', 'go_right', 'go_left', 'go_straight', 'go_right_or_straight', 'go_left_or_straight', 'keep_right', 'keep_left', 'roundabout', 'restriction_ends_overtaking', 'restriction_ends_overtaking_trucks']

In [23]:
print("Current yaml file contents:")
!cat /content/yolov5/data/gtsrb.yaml

Current yaml file contents:
train: /content/yolov5/GTSRB_YOLO/images/train
val: /content/yolov5/GTSRB_YOLO/images/val

nc: 43
names: ['20_speed', '30_speed', '50_speed', '60_speed', '70_speed', '80_speed', '80_lifted', '100_speed', '120_speed', 'no_overtaking_general', 'no_overtaking_trucks', 'right_of_way_crossing', 'right_of_way_general', 'give_way', 'stop', 'no_traffic_both', 'no_trucks', 'no_entry', 'danger', 'bend_left', 'bend_right', 'bend', 'uneven_road', 'slippery_road', 'road_narrows', 'construction', 'traffic_signal', 'pedestrian_crossing', 'school_crossing', 'cycles_crossing', 'snow', 'animals', 'restriction_ends', 'go_right', 'go_left', 'go_straight', 'go_right_or_straight', 'go_left_or_straight', 'keep_right', 'keep_left', 'roundabout', 'restriction_ends_overtaking', 'restriction_ends_overtaking_trucks']


In [24]:
import os
print("Number of images in train:", len(os.listdir('/content/yolov5/GTSRB_YOLO/images/train')))
print("Number of images in val:", len(os.listdir('/content/yolov5/GTSRB_YOLO/images/val')))

Number of images in train: 31367
Number of images in val: 7842


# Train the YOLOv5 model

In [ ]:
!python train.py --img 640 --batch 16 --epochs 50 --data gtsrb.yaml --weights yolov5s.pt --cache

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2024-12-09 02:15:45.718705: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-09 02:15:45.743371: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-09 02:15:45.750405: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to regi

# Evaluate the model

In [ ]:
!python val.py --weights /content/drive/MyDrive/rob535/project/best_93.pt --data gtsrb.yaml --img 640

Traceback (most recent call last):
  File "/content/yolov5/val.py", line 39, in <module>
    from models.common import DetectMultiBackend
  File "/content/yolov5/models/common.py", line 27, in <module>
    import ultralytics
  File "/usr/local/lib/python3.10/dist-packages/ultralytics/__init__.py", line 11, in <module>
    from ultralytics.models import NAS, RTDETR, SAM, YOLO, FastSAM, YOLOWorld
  File "/usr/local/lib/python3.10/dist-packages/ultralytics/models/__init__.py", line 6, in <module>
    from .sam import SAM
  File "/usr/local/lib/python3.10/dist-packages/ultralytics/models/sam/__init__.py", line 3, in <module>
    from .model import SAM
  File "/usr/local/lib/python3.10/dist-packages/ultralytics/models/sam/model.py", line 22, in <module>
    from .build import build_sam
  File "/usr/local/lib/python3.10/dist-packages/ultralytics/models/sam/build.py", line 19, in <module>
    from .modules.tiny_encoder import TinyViT
  File "/usr/local/lib/python3.10/dist-packages/ultralytics

# Test the model on new images

In [ ]:
# Assuming you have test images in GTSRB/GTSRB/Final_Test/Images
!python detect.py --weights /content/drive/MyDrive/rob535/project/best_93.pt --img 640 --conf 0.25 --source /content/yolov5/GTSRB/GTSRB/Final_Test/Images

Traceback (most recent call last):
  File "/content/yolov5/detect.py", line 38, in <module>
  File "/usr/local/lib/python3.10/dist-packages/torch/__init__.py", line 2486, in <module>
    from torch import _meta_registrations
  File "/usr/local/lib/python3.10/dist-packages/torch/_meta_registrations.py", line 10, in <module>
    from torch._decomp import (
  File "/usr/local/lib/python3.10/dist-packages/torch/_decomp/__init__.py", line 249, in <module>
    import torch._decomp.decompositions
  File "/usr/local/lib/python3.10/dist-packages/torch/_decomp/decompositions.py", line 15, in <module>
    import torch._prims as prims
  File "/usr/local/lib/python3.10/dist-packages/torch/_prims/__init__.py", line 825, in <module>
    log1p = _make_elementwise_unary_prim(
  File "/usr/local/lib/python3.10/dist-packages/torch/_prims/__init__.py", line 491, in _make_elementwise_unary_prim
    return _make_prim(
  File "/usr/local/lib/python3.10/dist-packages/torch/_prims/__init__.py", line 319, in _m

In [ ]:
!python detect.py --weights /content/drive/MyDrive/rob535/project/best_93.pt --source test_images --img 640 --conf 0.25 --benchmark

Traceback (most recent call last):
  File "/content/yolov5/detect.py", line 38, in <module>
    import torch
  File "/usr/local/lib/python3.10/dist-packages/torch/__init__.py", line 2486, in <module>
    from torch import _meta_registrations
  File "/usr/local/lib/python3.10/dist-packages/torch/_meta_registrations.py", line 10, in <module>
    from torch._decomp import (
  File "/usr/local/lib/python3.10/dist-packages/torch/_decomp/__init__.py", line 250, in <module>
    import torch._refs
  File "/usr/local/lib/python3.10/dist-packages/torch/_refs/__init__.py", line 6332, in <module>
    diagonal_copy = _make_copy_from_view(aten.diagonal)
  File "/usr/local/lib/python3.10/dist-packages/torch/_refs/__init__.py", line 2209, in _make_copy_from_view
    fn = out_wrapper()(aten_fn)
  File "/usr/local/lib/python3.10/dist-packages/torch/_prims_common/wrappers.py", line 259, in _out_wrapper
    sig = inspect.signature(fn)
  File "/usr/lib/python3.10/inspect.py", line 3254, in signature
    re

# Analyze results

In [25]:
import torch
from utils.general import check_img_size
from utils.torch_utils import select_device
from models.common import DetectMultiBackend

# Load model
device = select_device('')
# model = DetectMultiBackend('/content/drive/MyDrive/rob535/project/best_93.pt', device=device)
model = DetectMultiBackend('/content/drive/MyDrive/ROB535_group_project/best_93.pt', device=device)

# Test inference speed
with torch.no_grad():
    model(torch.zeros(1, 3, 640, 640).to(device))
    for _ in range(100):  # warmup
        model(torch.zeros(1, 3, 640, 640).to(device))

    import time
    times = []
    for _ in range(100):  # timing
        start = time.time()
        model(torch.zeros(1, 3, 640, 640).to(device))
        times.append(time.time() - start)

    print(f'Average inference time: {sum(times)/len(times)*1000:.2f}ms')
    print(f'FPS: {1/(sum(times)/len(times)):.2f}')

YOLOv5 🚀 v7.0-389-ge62a31b6 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
YOLOv5s summary: 224 layers, 7167184 parameters, 0 gradients


Average inference time: 7.70ms
FPS: 129.79
